# Create a Slice and Add/Remove SSH Keys



## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

In [ ]:
from plugins import Plugins
import traceback
try:
    Plugins.load()
except Exception as e:
    traceback.print_exc()

## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  


In [ ]:
slice_name = 'SSH-Keys'
site1 = fablib.get_random_site()
print(f"Sites: {site1}")

node1_name='Node1'
node2_name='Node2'

network_name='net1'
nic_name = 'nic1'
model_name = 'NIC_Basic'

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[1:]

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

net1 = slice.add_l2network(name=network_name, subnet=subnet)


hosts = [f"{site1.lower()}-w1", f"{site1.lower()}-w2", f"{site1.lower()}-w3"]

for h in hosts:

    # Node1
    node1 = slice.add_node(name=f"{h}-a", cores=16, ram=32, site=site1, image='docker_rocky_8', host=f"{h}.fabric-testbed.net")
    #node1.add_fabnet()

    iface1 = node1.add_component(model=model_name, name=nic_name).get_interfaces()[0]
    #iface1.set_mode('auto')
    net1.add_interface(iface1)

#Submit Slice Request
slice.submit();

### Create and add ssh keys

In [ ]:
slice = fablib.get_slice(slice_name)

In [ ]:
import paramiko

username = "rocky"

for node in slice.get_nodes():
    # Generate a new RSA key pair
    key = paramiko.RSAKey.generate(bits=2048)
    public_key_string = f"{key.get_name()} {key.get_base64()}"
    
    private_key_path = "id_rsa_test"
    with open(private_key_path, "w") as private_key_file:
        key.write_private_key(private_key_file)
        
    key_dict = {'key': public_key_string, 
                'comment': 'ephemerial key added via POA'}
    
    print(f"Adding ssh keys to node {node.get_name()}")
    node.perform_operational_action(operation="addkey", keys=[key_dict])
    print(f"Key added successfully to node {node.get_name()}")
    print("Validate ssh keys-----")
    try:
        stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`', 
                                      private_key_file=private_key_path)
    except Exception as e:
        print("Error:", str(e))
    print("Validation complete---")
    print(f"Remove ssh keys to node {node.get_name()}")
    node.perform_operational_action(operation="removekey", keys=[key_dict])
    print(f"Key removed successfully to node {node.get_name()}")
    print()
    
    

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()